In [1]:
#pip install requests

Note: you may need to restart the kernel to use updated packages.


In [6]:
import json

In [29]:
import requests
from datetime import datetime 

myurl = "https://api.powerbi.com/beta/5ee35505-eb8e-4929-937d-645df5013288/datasets/50ad263b-54e0-44ba-93e7-b00cbc39f84d/rows?key=vO%2BdZoHdf6qaU5fg86ZXyj8H%2B4sGYbjCaU6CIvKh8kdC7hnu24Rp38n7785Cvjr4ldnLMGUjL9WuMsZH0CfDsw%3D%3D"

date = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")

data= [
{
"temp_datetime" :date,
"temp_data" :40,
"gsr_datetime" :date,
"gsr_data" :20
}
]

r = requests.post(myurl, json.dumps(data))

In [14]:
data= [{'temp_data' : (100, 200)}]

print(json.dumps(data))

[{"temp_data": [100, 200]}]


In [22]:
from datetime import datetime 

print(datetime.now().strftime("%Y-%m-%dT%H:%M:%S"))

2021-07-19T13:21:02
